In [1]:
import pyspark
from pyspark.sql import *
spark = SparkSession.builder.appName('logistic_regression_fraud').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip install numpy --user' into the EC2 console.
from pyspark.ml.classification import LogisticRegression

In [2]:
# Use Spark to read in the Ecommerce Customers csv file. You can infer csv schemas. 
data = spark.read.csv("E:\INFOSYS 722\Assignments\Assignments\Iter4\Iteration4\CleanData.csv",inferSchema=True,header=True)

In [4]:
# Print data schema.
data.printSchema()

# Print data columns.
print(data.columns)

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

In [7]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [9]:
# Now we can assemble all of this as one vector in the features column. 
assembler = VectorAssembler(inputCols=['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],outputCol='features')

In [15]:
from pyspark.ml.classification import LogisticRegression

# Pipelines

A pipeline sets stages for different steps. Let's see an example of how to use pipelines.

In [13]:
from pyspark.ml import Pipeline

In [16]:
# Note that survived is a categorial variable but didn't require any transformation.
# That's because it's already in the format of 1's and 0's. 
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Class')

In [18]:
# Lists everything we want to do. Index data, encode data, assemble data and then pass in the actual model.
pipeline = Pipeline(stages=[assembler,log_reg_titanic])

In [30]:
# Train/test split. 
train_data, test_data = data.randomSplit([0.7,.3])

In [31]:
# Note pipeline. Call it as you would call a machine learning object.
fit_model = pipeline.fit(train_data)

In [32]:
# Transform test data. 
results = fit_model.transform(test_data)

In [33]:
# Evaluate the model using the binary classifer.
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Class')

In [27]:
# If we select the actual and predicted results, we can see that some predictions were correct while others were wrong.
results.select('Class','prediction').show()

+-----+----------+
|Class|prediction|
+-----+----------+
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
+-----+----------+
only showing top 20 rows



In [34]:
# We can then evaluate using AUC (area under the curve). AUC is linked to ROC.
AUC = my_eval.evaluate(results)

AUC

0.9410865008780378

# Data Visualisation

PySpark doesn't allow you to plot data directly. Instead, the simplest way to visualise data is to convert the DataFrame into an array, then use Python's extensive libraries (such as Matplotlib) to visualise the data appropriately. Because of this, the sky's the limit when it comes to data visualisation! Here's a simple example:

# Other Forms of Interpretation

In [35]:
# Not all interpretation has to be visualisations. You can also gain a lot of information with text.
# For example, here we're seeing how much variance our model could account for.
# According to this, the model got 164/209 correct. 
totalResults = results.select('Class','prediction')

correctResults = totalResults.filter(totalResults['Class'] == totalResults['prediction'])

countTR = totalResults.count()
print("Correct: " + str(countTR))

countTC = correctResults.count()
print("Total Correct: " + str(countTC))

Correct: 266
Total Correct: 250
